In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
!conda install -c weinbe58 quspin

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.



In [3]:
import quspin
from __future__ import print_function, division
import sys,os
from quspin.basis import spin_basis_general # Hilbert space spin basis
import numpy as np # generic math functions

Now, lets try and build our 2d antiferromagnetic Heisenberg model using:
\begin{equation}
\hat{H} = -\frac{1}{2} \sum_{j=1}^{N} \left( J_x \sigma_j^x \sigma_{j+1}^x + J_y \sigma_j^y \sigma_{j+1}^y + J_z \sigma_j^z \sigma_{j+1}^z \right)
\end{equation}


In [7]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spin_basis_1d # Hilbert space spin basis
import numpy as np # generic math functions

In [8]:
# First, practicing creating a 1d model
# Define the system size and coupling constants
L = 12  # system size
Jx = 1.0  # exchange coupling in the x-direction
Jy = 1.0  # exchange coupling in the y-direction
Jz = 1.0  # exchange coupling in the z-direction

# Set up the spin basis
basis=spin_basis_general(L,)

In [9]:
J_xx = [[-Jx/2.0, i, i + 1] for i in range(L-1)]  # OBC
J_yy = [[-Jy/2.0, i, i + 1] for i in range(L-1)]  # OBC
J_zz = [[-Jz/2.0, i, i + 1] for i in range(L-1)]  # OBC

static = [["xx", J_xx], ["yy", J_yy], ["zz", J_zz]]
dynamic = []

# compute the time-dependent Heisenberg Hamiltonian
H_XYZ = hamiltonian(static, dynamic, basis=basis, dtype=np.float64)


Hermiticity check passed!
Symmetry checks passed!


In [12]:
##### various exact diagonalisation routines #####
# calculate entire spectrum only
E=H_XYZ.eigvalsh()
print('entire energy spectrum: ', E)
# calculate full eigensystem
E,V=H_XYZ.eigh()
print('full eigensystem: ', E, V)
# calculate minimum and maximum energy only
Emin,Emax=H_XYZ.eigsh(k=2,which="BE",maxiter=1E4,return_eigenvectors=False)
print('minimum and maximum energy only: ', Emin, Emax)
# calculate the eigenstate closest to energy E_star
E_star = 0.0
E,psi_0=H_XYZ.eigsh(k=1,sigma=E_star,maxiter=1E4)
psi_0=psi_0.reshape((-1,))
print("eigenstate closest to energy E_star = 0.0. psi_0: ", psi_0)


entire energy spectrum:  [-5.5        -5.5        -5.5        ...  9.72229587  9.72229587
 10.28418127]
full eigensystem:  [-5.5        -5.5        -5.5        ...  9.72229587  9.72229587
 10.28418127] [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.88675135e-01  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.88675135e-01  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00 -5.46468092e-02  1.63509315e-01 ... -5.33326166e-44
   1.20352055e-52  1.86286490e-48]
 [ 0.00000000e+00 -5.46468092e-02  1.63509315e-01 ... -1.21277228e-43
   1.15725610e-51  2.10466709e-48]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
minimum and maximum energy only:  -5.4999999999999964 10.284181265681063
eigenstate closest to energy E_star = 0.0. psi_0:  [ 2.10901282e-18  2.32526252e-18  2.75

In [24]:
# now, for 2d system -- this is a 2x2 system
# initializing:
Lx = 2  # system size in the x-direction
Ly = 2  # system size in the y-direction
N_2d = Lx*Ly # total number of sites

###### setting up user-defined symmetry transformations for 2d lattice ######
s = np.arange(N_2d) # sites [ 0 ,1,2,....]
x = s%Lx # x positions for sites
y = s//Lx # y positions for sites
T_x = (x+1)%Lx + Lx*y # translation along x-direction
T_y = x +Lx*((y+1)%Ly) # translation along y-direction
P_x = x + Lx*(Ly-y-1) # reflection about x-axis
P_y = (Lx-x-1) + Lx*y # reflection about y-axis
Z = -(s+1) # spin inversion

# Set up the spin basis for a 2D system
basis_2d = spin_basis_general(N_2d,kxblock=(T_x, 0 ),kyblock=(T_y, 0 ), pxblock=(P_x, 0 ),pyblock=(P_y, 0 ),zblock=(Z, 0 ), pauli = True) # 2d - basis
# print info
print("Size of 2D H-space: {Ns:d}".format(Ns=basis_2d.Ns))

# defining site coupling ## -1 indicates antiferromagnetic
Jxx_2d=[[-1.0 ,i,T_x[i]] for i in range(N_2d)]+[[-1.0 ,i,T_y[i]] for i in range( N_2d)]
Jyy_2d=[[-1.0 ,i,T_x[i]] for i in range(N_2d)]+[[-1.0 ,i,T_y[i]] for i in range( N_2d)]
Jzz_2d=[[-1.0 ,i,T_x[i]] for i in range(N_2d)]+[[-1.0 ,i,T_y[i]] for i in range( N_2d)]


static = [["xx", Jxx_2d], ["yy", Jyy_2d], ["zz", Jzz_2d]]
dynamic = []

# compute the time-dependent Heisenberg Hamiltonian for a 2D system
H_XYZ_2D = hamiltonian(static, dynamic, basis=basis, dtype=np.float64)

##### various exact diagonalisation routines #####
# calculate entire spectrum only
E=H_XYZ_2D.eigvalsh()
print('entire energy spectrum: ', E)
# calculate full eigensystem
E,V=H_XYZ_2D.eigh()
print('eigenvectors: ', V)
# calculate minimum and maximum energy only
Emin,Emax=H_XYZ_2D.eigsh(k=2,which="BE",maxiter=1E4,return_eigenvectors=False)
print('minimum and maximum energy only: ', Emin, Emax)

# calculate the eigenstate closest to energy E_star
E_star = 0.01
E,psi_0=H_XYZ_2D.eigsh(k=1,sigma=E_star,maxiter=1E4)
psi_0=psi_0.reshape((-1,))
print("eigenstate closest to energy E_star = 0.0. psi_0: ", psi_0)


Size of 2D H-space: 5
Hermiticity check passed!
Symmetry checks passed!
entire energy spectrum:  [-8.00000000e+00 -8.00000000e+00 -8.00000000e+00 -8.00000000e+00
 -8.00000000e+00 -1.38522558e-15 -2.44591351e-16 -9.25613743e-17
  0.00000000e+00  2.77555756e-17  5.70417003e-17  4.07966641e-16
  8.00000000e+00  8.00000000e+00  8.00000000e+00  1.60000000e+01]
eigenvectors:  [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -5.00000000e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  7.07106781e-01
   0.00000000e+00  5.00000000e-01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -5.00000000e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+

<ipython-input-24-52e07c39698c>:18: GeneralBasisWarning: mappings for block kxblock and block pyblock produce the same symmetry.
  basis_2d = spin_basis_general(N_2d,kxblock=(T_x, 0 ),kyblock=(T_y, 0 ), pxblock=(P_x, 0 ),pyblock=(P_y, 0 ),zblock=(Z, 0 ), pauli = True) # 2d - basis
<ipython-input-24-52e07c39698c>:18: GeneralBasisWarning: mappings for block kyblock and block pxblock produce the same symmetry.
  basis_2d = spin_basis_general(N_2d,kxblock=(T_x, 0 ),kyblock=(T_y, 0 ), pxblock=(P_x, 0 ),pyblock=(P_y, 0 ),zblock=(Z, 0 ), pauli = True) # 2d - basis
